In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import emoji
# Permite desplegar el texto completo en Jupyter
pd.set_option('display.max_colwidth', -1)

In [2]:
# Lectura de CSV
data = pd.read_csv("tweets_es.csv")
#data = df.head(50) # Muestra los datos

## Quitar caracteres, link, @, Hashtag

In [3]:
data_noURL = data["content"].str.replace('\w+:\/\/\S+',"")
data_noUser = data_noURL.str.replace('@(\w+)',"")
data_noHashtag = data_noUser.str.replace('#(\w*)',"")
data_noEnter = data_noHashtag.str.replace('\n(w*)',"")
data_noAmp = data_noEnter.str.replace('&amp',"")
data_lower=data_noAmp.str.lower()

In [4]:
emoji_pattern = data_lower.str.replace("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", "")

In [5]:
data_noPuntuacion = emoji_pattern.str.replace('[^a-zA-Z ]+', '')
data_noPuntuacion = data_noPuntuacion.str.replace(" +"," ") # Reducir los espacios a solo 1
data_noRepeated = data_noPuntuacion.transform(lambda x: re.sub(r'(.)\1+', r'\1\1', str(x)))

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words("spanish")
stop_set = set(stop)
data_noStopwords = data_noRepeated.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_set)]))
#data_noStopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GIYELI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
import spacy
nlp = spacy.load('es')

In [9]:
#for token in nlp(data_noStopwords):
#    print(token.lemma_)

In [10]:
def lematizacion (tweet):
    #tokens = word_tokenize(tweet)
    resultado = []
    for token in nlp(tweet):
        #print(token.lemma_)
        resultado.append(token.lemma_)
    return " ".join(resultado)

In [11]:
data_noStopwords.head(10)

0    salgo da ms largoo                                                                  
1    libraras ayudar menos besos gracias                                                 
2    gracias mar                                                                         
3    off pensando regalito sinde va sgae van corruptos intento sacar conclusiones intento
4    conozco alguien q adicto drama ja ja ja suena d                                     
5    rt si amas alguien djalo libre si grita hombre mo                                   
6    toca grabacin dl especial navideomari crismas                                       
7    hoy asisitir madrid seminario estrategia espaola seguridad organizado faes          
8    buen da primero mandar abrazo grande miguel familia hoy podra ser da grandeza humana
9    escao listo empezar congreso                                                        
Name: content, dtype: object

In [12]:
data_Lemmatizer= data_noStopwords.apply(lambda x: lematizacion(x))
data_Lemmatizer.head(10)

0    salir dar milisegundo largoo                                                       
1    librar ayudar menos beso gracia                                                    
2    gracia mar                                                                         
3    off pensar regalito sinde ir sgae ir corrupto intentar sacar conclusión intentar   
4    conocer alguien q adicto drama ja ja ja sonar d                                    
5    rt si amar alguien djalo librar si gritar hombre mo                                
6    tocar grabacin decilitro especial navideomari crisma                               
7    hoy asisitir madrid seminario estrategia espaola seguridad organizar faes          
8    bueno dar 1 mandar abrazar grande miguel familia hoy pudrir ser dar grandeza humano
9    escao listar empezar congreso                                                      
Name: content, dtype: object

In [13]:
data["data_lemmatized"] = data_Lemmatizer
tweets_limpios = data[['data_lemmatized','sentiment']]
tweets_limpios.head(10)

,data_lemmatized,sentiment
0,salir dar milisegundo largoo,NONE
1,librar ayudar menos beso gracia,NEU
2,gracia mar,P
3,off pensar regalito sinde ir sgae ir corrupto intentar sacar conclusión intentar,N
4,conocer alguien q adicto drama ja ja ja sonar d,P
5,rt si amar alguien djalo librar si gritar hombre mo,NONE
6,tocar grabacin decilitro especial navideomari crisma,P
7,hoy asisitir madrid seminario estrategia espaola seguridad organizar faes,NONE
8,bueno dar 1 mandar abrazar grande miguel familia hoy pudrir ser dar grandeza humano,P
9,escao listar empezar congreso,P


In [14]:
tweets_limpios = tweets_limpios.loc[tweets_limpios['sentiment'] != "NONE"]
print("")

In [15]:
tweets_limpios["sentiment"] = tweets_limpios["sentiment"].replace(to_replace=["P","NEU","N"], value=[1,0,-1])

In [16]:
tweets_limpios.head(10)

,data_lemmatized,sentiment
1,librar ayudar menos beso gracia,0
2,gracia mar,1
3,off pensar regalito sinde ir sgae ir corrupto intentar sacar conclusión intentar,-1
4,conocer alguien q adicto drama ja ja ja sonar d,1
6,tocar grabacin decilitro especial navideomari crisma,1
8,bueno dar 1 mandar abrazar grande miguel familia hoy pudrir ser dar grandeza humano,1
9,escao listar empezar congreso,1
10,bdas em ira puente si ir dejeis llevar tableta pc luego orbyt momento decir milisegundo,1
11,sistema econmico q recortar dinero prestación social reforzar billn mediar d euro banco necesitar repensarse,1
12,caca d ajustar,-1


In [17]:
#tweets_limpios.to_csv('data_lemmatized_es.csv', index=None, header = True)

In [19]:
tweets_limpios["sentiment"].value_counts()

 1    3663
-1    3153
 0    1063
Name: sentiment, dtype: int64

In [1]:
import pandas as pd
test_df = pd.read_csv("tweets_es.csv")

In [2]:
test_df = test_df.loc[test_df['sentiment'] != "NONE"]

In [3]:
test_df["sentiment"] = test_df["sentiment"].replace(to_replace=["P", "NEU", "N"], value=[1, 0, -1])

In [4]:
test_df.head(10)

,tweetid,user,content,date,lang,sentiment
1,142389933619945473,CarmendelRiego,@PauladeLasHeras No te libraras de ayudar me/n...,2011-12-02T00:49:40,es,0
2,142391947707940864,CarmendelRiego,@marodriguezb Gracias MAR,2011-12-02T00:57:40,es,1
3,142416095012339712,mgilguerrero,"Off pensando en el regalito Sinde, la que se v...",2011-12-02T02:33:37,es,-1
4,142422495721562112,paurubio,Conozco a alguien q es adicto al drama! Ja ja ...,2011-12-02T02:59:03,es,1
6,142483342040907776,Carlos_Latre,Toca @crackoviadeTV3 . Grabación dl especial N...,2011-12-02T07:00:50,es,1
8,142494476051562496,nacho_uriarte,Buen día todos! Lo primero mandar un abrazo gr...,2011-12-02T07:45:05,es,1
9,142496796416016384,JuanraLucas,Desde el escaño. Todo listo para empezar #endi...,2011-12-02T07:54:19,es,1
10,142497735814287360,pedroj_ramirez,Bdías. EM no se ira de puente. Si vosotros os ...,2011-12-02T07:58:02,es,1
11,142499355360903168,sevillajordi,Un sistema económico q recorta dinero para pre...,2011-12-02T08:04:28,es,1
12,142504935853006848,Carlos_Latre,#programascambiados caca d ajuste,2011-12-02T08:26:38,es,-1


In [5]:
test_df.to_csv('tweets_es.csv', index=None, header = True)